In [8]:
# Desabilita o warning do request sem verificar certificado do servidor: 
# requests.get(url,verify=False) dispensa a verificaçao de certificado
import urllib3
urllib3.disable_warnings()

# Bibliotecas
import json # Utilizada para converter json em um dicionário em Python
import requests # Utilizada para fazer consultas em url
import pandas as pd # Utilizada para tabular dados
from datetime import datetime

In [15]:
url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
resp_dict = requests.get(url, verify=False).json()
mkt = resp_dict['response']['TrsrBondMkt']
mkt

{'opngDtTm': '2025-05-22T09:25:00',
 'clsgDtTm': '2025-05-23T05:00:00',
 'qtnDtTm': '2025-05-22T09:21:19.22',
 'stsCd': 1,
 'sts': 'Aberto'}

In [ ]:
# Verifica se mercado está aberto ou fechado
# ...existing code...
def status_mercado_df():
    url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
    resp_dict = requests.get(url, verify=False).json()
    mkt = resp_dict['response']['TrsrBondMkt']
    
    # Converte datas e horários para o formato desejado
    abertura_data = datetime.strptime(mkt['opngDtTm'][:10], "%Y-%m-%d").strftime("%d/%m/%Y")
    abertura_hora = mkt['opngDtTm'][11:16]
    fechamento_data = datetime.strptime(mkt['clsgDtTm'][:10], "%Y-%m-%d").strftime("%d/%m/%Y")
    fechamento_hora = mkt['clsgDtTm'][11:16]
    status = mkt['sts']
    
    df = pd.DataFrame([{
        'Status': status,
        'Data Abertura': abertura_data,
        'Horário de Abertura': abertura_hora,
        'Data Fechamento': fechamento_data,
        'Horário de Fechamento': fechamento_hora        
    }])
    return df


def consultaTD(op,tp):
    """
    op:'C','V','' para Compra, Venda ou ambos
    tp:'S','P','I', '' para Selic, Prefixado, IPCA ou todos
    """
    op = op
    tp = tp
    
    url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
    resp_dict = requests.get(url,verify=False).json()
    
    n_titulos = len(resp_dict['response']['TrsrBdTradgList'])
    
    tipos = []
    nomes = []
    tx_compras = []
    tx_vendas = []
    p_compras = []
    p_vendas = []
    vencimentos = []
    
    
    for i in range(n_titulos):
        tipos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['FinIndxs']['nm']) 
        nomes.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm'])
        vencimentos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['mtrtyDt'])
        tx_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulInvstmtRate'])
        p_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrInvstmtVal'])
        tx_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulRedRate'])
        p_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrRedVal']) 
    
    
    df = pd.DataFrame()
    df['Tipo'] = tipos
    df['Título'] = nomes
    df['Vencimento'] = pd.to_datetime(vencimentos)
    df['Rentabilidade (Compra)'] = tx_compras
    df['Preço R$ (Compra)'] = p_compras    
    df['Rentabilidade (Venda)'] = tx_vendas
    df['Preço R$ (Venda)'] = p_vendas
    
    if tp == 'S':
        df = df.iloc[df[df['Tipo']=='SELIC'].index]
    elif tp == 'P':
        df = df.iloc[df[df['Tipo']=='PREFIXADO'].index]
    elif tp == 'I':
        df = df.iloc[df[df['Tipo']=='IPCA'].index]
    else:
        df = df
        
    if op == 'C':
        return df[df['Preço R$ (Compra)']!=0].drop(df.columns[[5, 6]],axis=1)
    elif op == 'V':
        return df.drop(df.columns[[3, 4]],axis=1)
    else:
        return df

In [18]:
status_mercado_df()

,Status,Data Abertura,Horário de Abertura,Data Fechamento,Horário de Fechamento
0,Aberto,22/05/2025,09:25,23/05/2025,05:00


In [4]:
consultaTD('','')

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
0,IPCA,Tesouro IPCA+ 2026,2026-08-15,0.0000,0.00,9.3400,4038.35
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.4100,3401.66,7.5300,3385.31
2,IPCA,Tesouro IPCA+ 2035,2035-05-15,0.0000,0.00,7.3600,2229.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.0900,1597.89,7.2100,1570.38
4,IPCA,Tesouro IPCA+ 2045,2045-05-15,0.0000,0.00,7.2700,1118.82
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.9900,828.62,7.1100,805.33
6,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01,0.0000,0.00,14.1600,982.51
7,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01,0.0000,0.00,13.9700,934.99
8,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01,0.0000,0.00,14.2800,885.97
9,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01,0.0000,0.00,14.2300,855.34
